In [1]:
import pandas as pd

In [2]:
data_df = pd.read_csv('./train_data.csv')

## Day of Week

In [3]:
DAY_OF_WEEK_df = data_df.groupby('DAY_OF_WEEK').count()['QUARTER'].to_frame()
DAY_OF_WEEK_df.columns = ['count']
# num_delays : total number of delays in training set
num_delays = DAY_OF_WEEK_df['count'].sum()
DAY_OF_WEEK_df['probability'] = DAY_OF_WEEK_df['count'] / num_delays

In [4]:
DAY_OF_WEEK_df

,count,probability
DAY_OF_WEEK,,
1,128,0.165375
2,106,0.136951
3,93,0.120155
4,131,0.169251
5,138,0.178295
6,70,0.090439
7,108,0.139535
